In [1]:
import pandas as pd
from pycaret.regression import *
from pycaret.regression import load_model, predict_model
import datetime
import os

In [2]:
if not os.path.isdir('trained_models'):
    os.mkdir('trained_models')

In [3]:
# Load Redfin housing data
df_data = pd.read_csv("HousingRecommenderFinalDataset.csv")
df_data['date'] = pd.to_datetime(df_data['period_end'], format='%Y-%m-%d')

In [4]:
# Filter to retain All Residential data
df_res = df_data[df_data['property_type'] == 'All Residential']
df_res.reset_index(drop=True, inplace=True)

In [5]:
columns = ['date', 'county', 'state_code', 'median_ppsf']
df_res = pd.DataFrame(df_res, columns=columns)

In [6]:
df_res['county_state'] = df_res[['county', 'state_code']].apply(lambda x: '-'.join(x), axis=1)
df_res.drop(['county', 'state_code'], axis=1, inplace=True)

In [7]:
# extract features from date
df_res['month'] = [i.month for i in df_res['date']]
df_res['year'] = [i.year for i in df_res['date']]

In [8]:
ct = datetime.datetime.now()
print("current time:-", ct)

current time:- 2022-04-06 09:11:41.626220


In [9]:
counties = df_res['county_state'].unique()
    
all_results = []
final_model = {}

In [10]:
len(counties)

1860

In [70]:
for county in counties:
    
    df_county = df_res[df_res['county_state'] == county]
    
    # initialize setup from pycaret.regression
    s = setup(df_county, target = 'median_ppsf', train_size = 0.80,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ['date', 'county_state'],
              numeric_features = ['month', 'year'],
              categorical_features = ['month'],
              silent = True, verbose = False, session_id = 123)
    
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=False)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['county_state'] = str(county)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[county] = f
    
     # save transformation pipeline and model as pickle file 
    save_model(f, model_name='trained_models/' + str(county), verbose=False)

In [27]:
county

'Teller County-CO'

In [71]:
len(all_results)

1856

In [10]:
ct = datetime.datetime.now()
print("current time:-", ct)

current time:- 2022-04-05 22:17:21.340296


In [72]:
concat_results = pd.concat(all_results,axis=0)
print(concat_results.shape[0])

1820


In [75]:
concat_results

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),county_state
rf,Random Forest Regressor,3.6409,20.7104,4.5442,0.8960,0.0581,0.0469,0.2267,Morgan County-AL
gbr,Gradient Boosting Regressor,1.8069,5.9234,2.3252,0.9672,0.0246,0.0196,0.0400,Tulsa County-OK
lr,Linear Regression,18.9813,672.7615,25.2427,0.7624,0.0942,0.0747,0.0067,Hood River County-OR
omp,Orthogonal Matching Pursuit,27.3163,1564.9580,37.4556,0.0231,0.4271,0.4008,0.0167,Lamar County-MS
et,Extra Trees Regressor,3.3706,73.4086,6.1989,0.9254,0.0497,0.0296,0.1667,Pasco County-FL
...,...,...,...,...,...,...,...,...,...
ada,AdaBoost Regressor,41.0765,2380.7367,41.0765,NaN,0.3824,0.2998,0.0633,Unicoi County-TN
et,Extra Trees Regressor,19.3259,600.3481,19.3259,NaN,0.4292,0.5966,0.4633,Chester County-TN
et,Extra Trees Regressor,18.0819,511.2798,18.0819,NaN,0.4397,0.3542,0.4133,Lanier County-GA
ada,AdaBoost Regressor,4.7259,43.4967,4.7259,NaN,0.0585,0.0563,0.2300,Wayne County-MO


In [74]:
concat_results.to_csv("best_model.csv")

In [88]:
future_dates = pd.date_range(start='2022-01-01', end = '2024-01-01', freq = 'M')

In [89]:
len(future_dates)

24

In [90]:
df_future = pd.DataFrame()
df_future['date'] = future_dates
df_future['month'] = [i.month for i in df_future['date']]
df_future['year'] = [i.year for i in df_future['date']]


In [91]:
df_future.head()

,date,month,year
0,2022-01-31,1,2022
1,2022-02-28,2,2022
2,2022-03-31,3,2022
3,2022-04-30,4,2022
4,2022-05-31,5,2022


In [92]:
df_predict = []

for county in counties:
    if os.path.isfile('trained_models/' + str(county) + '.pkl'):
        l = load_model('trained_models/' + str(county), verbose=False)
        p = predict_model(l, data=df_future)
        p['time_series'] = county
        df_predict.append(p)
    


In [96]:
df_forecast = pd.concat(df_predict, axis=0)

In [97]:
df_forecast.shape

(43608, 5)

In [98]:
df_forecast.head()

,date,month,year,Label,time_series
0,2022-01-31,1,2022,100.563884,Morgan County-AL
1,2022-02-28,2,2022,98.752071,Morgan County-AL
2,2022-03-31,3,2022,99.923069,Morgan County-AL
3,2022-04-30,4,2022,104.868045,Morgan County-AL
4,2022-05-31,5,2022,108.512793,Morgan County-AL


In [99]:
df_forecast[['county', 'state']] = df_forecast['time_series'].str.split('-', 1, expand=True)
df_forecast = df_forecast.drop(columns=['time_series'])

df_forecast = df_forecast.rename(columns={"Label": "median_ppsf"})

df_forecast.head()

,date,month,year,median_ppsf,county,state
0,2022-01-31,1,2022,100.563884,Morgan County,AL
1,2022-02-28,2,2022,98.752071,Morgan County,AL
2,2022-03-31,3,2022,99.923069,Morgan County,AL
3,2022-04-30,4,2022,104.868045,Morgan County,AL
4,2022-05-31,5,2022,108.512793,Morgan County,AL


In [101]:
df_forecast

,date,month,year,median_ppsf,county,state
0,2022-01-31,1,2022,100.563884,Morgan County,AL
1,2022-02-28,2,2022,98.752071,Morgan County,AL
2,2022-03-31,3,2022,99.923069,Morgan County,AL
3,2022-04-30,4,2022,104.868045,Morgan County,AL
4,2022-05-31,5,2022,108.512793,Morgan County,AL
...,...,...,...,...,...,...
19,2023-08-31,8,2023,111.425594,Monroe County,OH
20,2023-09-30,9,2023,80.487974,Monroe County,OH
21,2023-10-31,10,2023,106.644334,Monroe County,OH
22,2023-11-30,11,2023,56.714675,Monroe County,OH


In [105]:
df_forecast.to_csv("ppsf_forecast.csv", index = False)